# 用户指南，第5章：嵌套列表，函数与递归
[原文链接](https://web.mit.edu/music21/doc/usersGuide/usersGuide_05_listsOfLists.html#usersguide-05-listsoflists)

[上一章](04.ipynb)我们介绍了Python列表，music21 Stream与列表的相似之处，以及如何把音符存入Stream，查看其时刻，并用`.show()`函数将Stream以MusicXML或文本形式显示出来。最后，把Stream存入Stream，看起来像是一个完美的技巧，直到我们发现无法获取到内层Stream中的音符。

本章中，我们会继续介绍怎么使用嵌套Stream。我们会先从嵌套列表开始。如果你有一定Python经验，则可以跳过本章。
## 嵌套列表
列表（类似于其他编程语言的数组）可以存储各种对象，包括列表。我们先创建两个列表：

In [1]:
listA = [10, 20, 30]
listB = [1, 2, 3, listA]

当我们访问`listB`时，我们会看到`listA`在里面：

In [2]:
listB

[1, 2, 3, [10, 20, 30]]

注意listB的长度是4个元素，而不是6：

In [3]:
len(listB)

4

这是因为`listB`的第四个元素（`listB[3]`）本身也是一个列表，`listA`

In [4]:
listB[3]

[10, 20, 30]

In [5]:
listB[3] is listA

True

所以，如果要获取`listA`的第三个元素，有一种简单的办法：

In [6]:
listA[2]

30

但是我们也可以认为`30`也是`listB`的第四个元素的第三个元素。所以可以这样写：

In [7]:
listB[3][2]

30

另外，既然这它们都是对应列表的最后一个元素，我们也可以写：

In [8]:
listB[-1][-1]

30

意思是：“获取`listB`的最后一个元素的最后一个元素”

但是如果我们需要知道`listB`中所有结构里面存储的所有数值，甚至嵌套列表中的数值，怎么办？以下代码是不行的：

In [9]:
for number in listB:
    print(number)

1
2
3
[10, 20, 30]


我们得检测`listB`中的每个元素是确实是一个数值，还是一个列表。如果是一个列表，则需要访问列表中的每个元素并输出。以下代码可以做到这一点。

In [10]:
for thing in listB:
    if isinstance(thing, list):
        for number in thing:
            print(number)
    else:
        print(thing)

1
2
3
10
20
30


以上代码是怎么运行的呢？首先，我们一件一件地查看`listB`中的每一样“东西”（`thing`）（我们暂时不知道它是数值还是列表）。然后下一行`if isinstance(thing, list):`检查它是不是列表。如果是列表，就进入内层循环，查看内层列表的每一个元素。但是如果它不是列表，则进入`else`分支，并直接输出元素。

（这里我们假定`listB`中只有两种元素：数值和其他的列表。）如果出现错误，请检查有没有漏掉冒号`:`和缩进。

## 函数与递归
如果再套一层列表：

In [11]:
listC = [100, 200, 300, listB]

由于`listB`里面包含`listA`，这是一个三层嵌套列表：

In [12]:
listC

[100, 200, 300, [1, 2, 3, [10, 20, 30]]]

如果要输出`listC`中的所有数值，可以写一大段丑陋的代码来逐层展开列表：

In [13]:
for thing in listC:
    if isinstance(thing, list):
        for innerThing in thing:
            if isinstance(innerThing, list):
                for number in innerThing:
                    print(number)
            else:
                print(innerThing)
    else:
        print(thing)

100
200
300
1
2
3
10
20
30


但是你应该已经猜到了，如果继续套一层：` listD = [4, 5, listC, 6, 7] `，上面的代码就又不能用了。所以要使用一种叫“递归”的手段。

In [14]:
def flatPrint(myList):
    for thing in myList:
        if isinstance(thing, list):
            flatPrint(thing)
        else:
            print(thing)

以上代码中，创建了`flatPrint`函数。该函数的功能是：深入每一层列表并输出其中内容。

In [15]:
flatPrint(listC)

100
200
300
1
2
3
10
20
30


这段代码又是怎么运行的呢？

`def`关键字的意思是“定义”一个新的函数，其后面跟着函数名——等下调用函数的时候要用到它。（我们将把一个嵌套结构转为线性结构的操作称为“扁平化”（`flatten`），就像踩扁纸盒子一样。由于这是一种扁平化操作，同时输出其中内容，`flatPrint`是一个不错的函数名。注意`python`中无论是变量名还是函数名都区分大小写，所以`flatPrint`和`flatprint`、`Flatprint`、`FlAtPrInT`各不相同）

在"flatPrint"之后，括号里面是变量名`mylist`。注意我们还没用过`myList`这个变量名——它还不存在。那么这里`myList`的意思是：当我们调用函数`flatPrint`的时候，无论这个列表叫什么名字，在`flatPrint`里面，我们都叫它`myList`。所以我们可以`flatPrint(listC)`，在这个`flatPrint`里面`listC`就被叫做`myList`。

下面的例子可以更清晰地解释这一点。`squareMe`接受一个数值作为自变量，并打印其平方：

In [16]:
def squareMe(number):
    print(number * number)

现在可以尝试：

In [17]:
squareMe(10)

100


In [18]:
squareMe(2.5)

6.25


In [19]:
pi = 3.14
squareMe(pi)

9.8596


在最后一个例子中注意两件事。第一点，π并非精确地等于3.14——我们都知道这一点。我只是想让教室里的数学老师别冲塔。第二点，我们将`pi`作为自变量传入`squareMe`函数，但是在函数中我们不会写`print(pi * pi)`，而是用`number`表示`pi`。

在函数最后，既可以把结果`print`出来，也可以用`return`把结果返回到主函数中，以便用于后续其他处理。例如这里的`cubeMe`函数：

In [20]:
def cubeMe(number):
    return number * number * number

这里就可以把`cubeMe`的值赋给另一个变量：

In [21]:
x = cubeMe(2)
x

8

In [22]:
y = cubeMe(x)
y

512

注意：如果要计算`x=cubeMe(2)`,`y=cubeMe(x)`可以直接把第一行代入第二行，并作一步`cubeMe(cubeMe(2))`：

In [23]:
y = cubeMe(cubeMe(2))
y

512

所以，与`print`相比，`return`功能更强大。在介绍完`flatPrint`之后，大部分的函数都将使用`return`而不是`print`。

回到`flatPrint`函数：

In [24]:
def flatPrint(myList):              # 1
    for thing in myList:            # 2
        if isinstance(thing, list): # 3
            flatPrint(thing)        # 4
        else:                       # 5
            print(thing)            # 6

我们来一行一行地分析

第1行：定义了`flatPrint`函数，接受一个参数叫`myList`。

第2行：对`myList`中的每个元素，都取出来，叫它`thing`。处理完当前的`thing`之后，程序就会跳回这一行，处理下一个`thing`

第3行：检查`thing`是不是列表。如果是的话，运行第4行。否则跳转到第5行

第4行：我们已经知道`thing`是列表。那么我们怎么打印一个列表（其中还有可能有列表）呢？还是用`flatPrint`！像这样在函数内部调用自己叫做*递归*。这是一种有效手段，我们在music21中会经常用到。

第5行：如果第3行发现`thing`不是列表，就跳转到这里，并向下执行第6行

第6行：简单地打印出`thing`，因为我们知道它是数值。

警告：不像其他的编程语言（Java、C等），python从来不会检查你传入`flatPrint`的变量是列表。所以你可以尝试`flatPrint(30)`。由于30不是列表，将会在函数里面报错：

In [25]:
flatPrint(30)

TypeError: 'int' object is not iterable

有关Python数据结构的更多信息（列表、嵌套列表等），可以观看谷歌Python教程，尤其是第2课

## 总结
本章，我们介绍了如何处理嵌套列表。这将对下面介绍music21中如何处理嵌套Stream有帮助。我们还学习了定义函数与递归。下一章中，我们将把这一切用于music21中：[用户指南，第6章：流(II)：分层，递归与扁平化](06.ipynb)